In [1]:
# Sentiment-Based Return Prediction Project
# Team Group X

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from scipy import stats
import pickle
import warnings
warnings.filterwarnings('ignore')

# Part 1: Load and Explore the Data
print("Loading data...")

# Replace these paths with the actual data file paths
sentiment_data_path = "sentiment_train_2017_2021.csv"
return_data_path = "return_train_2017_2021.csv"

# Load the data
sentiment_data = pd.read_csv(sentiment_data_path)
return_data = pd.read_csv(return_data_path)

# Display basic information about the data
print("Sentiment data shape:", sentiment_data.shape)
print("Return data shape:", return_data.shape)

# Convert Date columns to datetime
# For sentiment_data, create a 'Date' column from a timestamp if it doesn't already exist.
if 'Date' not in sentiment_data.columns:
    # Prefer using 'Received_Time' if available, else fall back to 'Post_Time'
    if 'Received_Time' in sentiment_data.columns:
        # Optionally, convert to a specific timezone and then get the date.
        sentiment_data['Date'] = pd.to_datetime(sentiment_data['Received_Time']).dt.floor('D')
    elif 'Post_Time' in sentiment_data.columns:
        sentiment_data['Date'] = pd.to_datetime(sentiment_data['Post_Time']).dt.floor('D')
    else:
        raise KeyError("No 'Date', 'Received_Time' or 'Post_Time' column found in sentiment data.")
else:
    sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date'])

# For the returns data, convert the 'Date' column to datetime.
return_data['Date'] = pd.to_datetime(return_data['Date'])

# Display head of both datasets
print("\nSentiment data sample:")
print(sentiment_data.head())

print("\nReturn data sample:")
print(return_data.head())

# Check for missing values
print("\nMissing values in sentiment data:")
print(sentiment_data.isnull().sum())

print("\nMissing values in return data:")
print(return_data.isnull().sum())


# Part 2: Feature Engineering
print("\n--- Feature Engineering ---")

def engineer_features(sentiment_data, return_data=None, training=True):
    """
    Engineer features from sentiment data.
    
    Parameters:
    -----------
    sentiment_data : DataFrame
        The Reddit sentiment data
    return_data : DataFrame, optional
        The stock return data (used during training)
    training : bool
        Whether this is for training or prediction
        
    Returns:
    --------
    features_df : DataFrame
        DataFrame with engineered features
    """
    # Make a copy to avoid modifying the original data
    sentiment = sentiment_data.copy()
    
    # Group by Date and Ticker
    grouped = sentiment.groupby(['Date', 'Ticker'])
    
    # Create a list to store daily features for each ticker
    daily_features = []
    
    # Process each day and ticker combination
    for (date, ticker), group in grouped:
        # Skip if too few posts (need at least 3 posts for meaningful stats)
        if len(group) < 3:
            continue
            
        # 1. Basic sentiment statistics
        mean_sentiment = group['Sentiment'].mean()
        median_sentiment = group['Sentiment'].median()
        std_sentiment = group['Sentiment'].std()
        min_sentiment = group['Sentiment'].min()
        max_sentiment = group['Sentiment'].max()
        
        # Handle NaN in std calculation for single posts
        if pd.isna(std_sentiment):
            std_sentiment = 0
            
        # 2. Volume indicators
        post_count = len(group)
        log_post_count = np.log1p(post_count)
        
        # 3. Probability-based features
        mean_prob_pos = group['Prob_POS'].mean()
        mean_prob_neg = group['Prob_NEG'].mean()
        mean_prob_ntr = group['Prob_NTR'].mean()
        
        # Sentiment certainty (max probability regardless of class)
        max_probs = group[['Prob_POS', 'Prob_NEG', 'Prob_NTR']].max(axis=1)
        mean_certainty = max_probs.mean()
        
        # Entropy of probability distribution (measure of ambiguity)
        def entropy(row):
            probs = [row['Prob_POS'], row['Prob_NEG'], row['Prob_NTR']]
            # Filter out zero probabilities to avoid log(0)
            probs = [p for p in probs if p > 0]
            return -sum(p * np.log(p) for p in probs)
        
        mean_entropy = group.apply(entropy, axis=1).mean()
        
        # 4. Author-based features
        unique_authors = group['Author'].nunique()
        author_ratio = unique_authors / post_count  # Ratio of unique authors to total posts
        
        # 5. Sentiment polarity and distribution
        pos_ratio = (group['Sentiment'] > 0).mean()  # Ratio of positive sentiment
        neg_ratio = (group['Sentiment'] < 0).mean()  # Ratio of negative sentiment
        ntr_ratio = (group['Sentiment'] == 0).mean()  # Ratio of neutral sentiment
        
        # Check if there are enough samples for skewness calculation
        if len(group) >= 3:
            sentiment_skew = group['Sentiment'].skew()
            if pd.isna(sentiment_skew):
                sentiment_skew = 0
        else:
            sentiment_skew = 0
            
        # 6. Weighted sentiment
        weighted_sentiment = (group['Sentiment'] * group['Prob_POS']).sum() / group['Prob_POS'].sum() \
            if group['Prob_POS'].sum() > 0 else mean_sentiment
            
        # Collect all features in a dictionary
        features = {
            'Date': date,
            'Ticker': ticker,
            'mean_sentiment': mean_sentiment,
            'median_sentiment': median_sentiment,
            'std_sentiment': std_sentiment,
            'min_sentiment': min_sentiment,
            'max_sentiment': max_sentiment,
            'sentiment_range': max_sentiment - min_sentiment,
            'post_count': post_count,
            'log_post_count': log_post_count,
            'mean_prob_pos': mean_prob_pos,
            'mean_prob_neg': mean_prob_neg,
            'mean_prob_ntr': mean_prob_ntr,
            'mean_certainty': mean_certainty,
            'mean_entropy': mean_entropy,
            'unique_authors': unique_authors,
            'author_ratio': author_ratio,
            'pos_ratio': pos_ratio,
            'neg_ratio': neg_ratio,
            'ntr_ratio': ntr_ratio,
            'sentiment_skew': sentiment_skew,
            'weighted_sentiment': weighted_sentiment
        }
        
        daily_features.append(features)
    
    # Convert to DataFrame
    features_df = pd.DataFrame(daily_features)
    
    if len(features_df) == 0:
        return pd.DataFrame()  # Return empty DataFrame if no features were created
        
    # If this is for training, merge with return data to get target variable
    if training and return_data is not None:
        # Merge features with next-day returns
        features_df = pd.merge(
            features_df,
            return_data[['Date', 'Ticker', 'Return']],
            on=['Date', 'Ticker'],
            how='left'
        )
        
        # Check if we have any missing returns and drop them
        missing_returns = features_df['Return'].isnull().sum()
        if missing_returns > 0:
            print(f"Warning: {missing_returns} rows have missing returns and will be dropped.")
            features_df = features_df.dropna(subset=['Return'])
    
    return features_df

# Engineer features for training data
print("Engineering features...")
features_df = engineer_features(sentiment_data, return_data, training=True)

print("Features dataframe shape:", features_df.shape)
print("\nFeatures sample:")
print(features_df.head())

# Display statistics of engineered features
print("\nEngineered features statistics:")
features_stats = features_df.describe()
print(features_stats)

# Convert Return column from percentage string to float if needed
if 'Return' in features_df.columns and features_df['Return'].dtype == 'object':
    features_df['Return'] = features_df['Return'].str.rstrip('%').astype('float') / 100.0

# Then proceed with the correlation analysis
numeric_features_df = features_df.select_dtypes(include=['float64', 'int64', 'float32', 'int32'])

## Analyze feature correlations with returns
print("\nFeature correlations with returns:")
# Drop non-numeric columns before calculating correlations
numeric_features_df = features_df.drop(['Date', 'Ticker'], axis=1)
corr_with_return = numeric_features_df.corr()['Return'].sort_values(ascending=False)
print(corr_with_return)

# Visualize top 10 feature correlations
plt.figure(figsize=(12, 6))
corr_with_return.drop('Return').nlargest(10).sort_values().plot(kind='barh')
plt.title('Top 10 Feature Correlations with Next-Day Returns')
plt.tight_layout()
plt.show()

# Part 3: Model Development
print("\n--- Model Development ---")

# Prepare data for modeling
X = features_df.drop(['Date', 'Ticker', 'Return'], axis=1)
y = features_df['Return']

# Check for and handle any remaining missing values
print("\nChecking for missing values in features:")
print(X.isnull().sum().sum())

# Fill any remaining NaN values with mean
if X.isnull().sum().sum() > 0:
    X = X.fillna(X.mean())

# Split data for training and validation
# Use time-based split to avoid look-ahead bias
train_cutoff = pd.to_datetime('2020-07-01')  # Use first ~3.5 years for training, last ~1.5 years for validation
train_mask = features_df['Date'] < train_cutoff
val_mask = features_df['Date'] >= train_cutoff

X_train, X_val = X[train_mask], X[val_mask]
y_train, y_val = y[train_mask], y[val_mask]
dates_val = features_df[val_mask]['Date']
tickers_val = features_df[val_mask]['Ticker']

print(f"Training data: {X_train.shape[0]} samples")
print(f"Validation data: {X_val.shape[0]} samples")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Try multiple models and compare their performance
models = {
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)
}

# Function to evaluate models
def evaluate_model(name, model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    # Calculate regression metrics
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    # Calculate directional accuracy
    directional_acc = np.mean((y_val > 0) == (y_pred > 0))
    
    # Create DataFrame to simulate portfolio performance
    val_df = pd.DataFrame({
        'Date': dates_val.values,
        'Ticker': tickers_val.values,
        'Actual_Return': y_val.values,
        'Predicted_Return': y_pred
    })
    
    # Rank predictions within each day
    val_df['Rank'] = val_df.groupby('Date')['Predicted_Return'].rank(ascending=False)
    
    # Create long-short portfolio (long top 10%, short bottom 10%)
    daily_counts = val_df.groupby('Date').size()
    val_df['Decile'] = val_df.groupby('Date')['Rank'].apply(
        lambda x: pd.qcut(x, 10, labels=False, duplicates='drop')
    )
    
    # Portfolio returns
    top_decile = val_df[val_df['Decile'] == 9]  # Top 10%
    bottom_decile = val_df[val_df['Decile'] == 0]  # Bottom 10%
    
    top_returns = top_decile.groupby('Date')['Actual_Return'].mean()
    bottom_returns = bottom_decile.groupby('Date')['Actual_Return'].mean()
    ls_returns = top_returns - bottom_returns  # Long-short returns
    
    # Portfolio performance metrics
    port_return = ls_returns.mean()
    port_volatility = ls_returns.std()
    sharpe = port_return / port_volatility if port_volatility > 0 else 0
    
    # Print results
    print(f"\n{name} Results:")
    print(f"RMSE: {rmse:.6f}")
    print(f"MAE: {mae:.6f}")
    print(f"R²: {r2:.6f}")
    print(f"Directional Accuracy: {directional_acc:.6f}")
    print(f"Portfolio Mean Daily Return: {port_return:.6f}")
    print(f"Portfolio Sharpe Ratio: {sharpe:.6f}")
    
    return {
        'model': model,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'directional_acc': directional_acc,
        'port_return': port_return,
        'port_volatility': port_volatility,
        'sharpe': sharpe
    }

# Evaluate all models
results = {}
for name, model in models.items():
    print(f"Training and evaluating {name}...")
    results[name] = evaluate_model(name, model, X_train_scaled, y_train, X_val_scaled, y_val)

# Find the best model based on portfolio Sharpe ratio
best_model_name = max(results.keys(), key=lambda k: results[k]['sharpe'])
best_model = results[best_model_name]['model']
print(f"\nBest model based on Sharpe ratio: {best_model_name}")

# Feature importance analysis for the best model
if best_model_name == 'Random Forest' or best_model_name == 'XGBoost':
    feature_importance = pd.Series(best_model.feature_importances_, index=X.columns)
    top_features = feature_importance.nlargest(10)
    
    plt.figure(figsize=(12, 6))
    top_features.sort_values().plot(kind='barh')
    plt.title(f'Top 10 Feature Importance ({best_model_name})')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 features by importance:")
    for feature, importance in top_features.items():
        print(f"{feature}: {importance:.4f}")

# Retrain the best model on all data
print("\nRetraining the best model on all data...")
best_model.fit(scaler.transform(X), y)

# Create model_info dictionary for the prediction function
model_info = {
    'model': best_model,
    'scaler': scaler,
    'feature_names': X.columns.tolist(),
    'model_type': best_model_name
}

# Save the model info for the prediction function
with open('model_info.pkl', 'wb') as f:
    pickle.dump(model_info, f)

print("Model saved successfully!")

# Save feature engineering function for use in prediction
def save_functions():
    # Create a dictionary with necessary functions
    functions = {
        'engineer_features': engineer_features
    }
    
    # Save the functions
    with open('functions.pkl', 'wb') as f:
        pickle.dump(functions, f)
    
    print("Functions saved successfully!")

save_functions()

# Part 4: Sample Prediction Function Demonstration
print("\n--- Sample Prediction Demonstration ---")

def predict_returns_demo(model_info, sentiment_data_today, stock_universe_today, historical_data=None):
    """
    Demo of prediction function to show how it works
    """
    # Load the engineer_features function
    with open('functions.pkl', 'rb') as f:
        functions = pickle.load(f)
    
    engineer_features = functions['engineer_features']
    
    # Extract model components
    model = model_info['model']
    scaler = model_info['scaler']
    feature_names = model_info['feature_names']
    
    # Engineer features for today's data
    features_today_df = engineer_features(sentiment_data_today, training=False)
    
    # Create a DataFrame for all tickers in the universe
    predictions_df = pd.DataFrame({'Ticker': stock_universe_today})
    
    # Initialize predicted returns to 0 (neutral prediction)
    predictions_df['Predicted_Return'] = 0.0
    
    # Update predictions for tickers with sentiment data
    if not features_today_df.empty:
        # Select tickers in universe that have features
        valid_tickers = features_today_df['Ticker'].unique()
        valid_tickers = [t for t in valid_tickers if t in stock_universe_today]
        
        if valid_tickers:
            # Filter features for valid tickers
            valid_features = features_today_df[features_today_df['Ticker'].isin(valid_tickers)]
            
            # Prepare features for prediction
            X_pred = valid_features[feature_names] if all(f in valid_features.columns for f in feature_names) else pd.DataFrame(columns=feature_names)
            
            # Handle missing features
            for feature in feature_names:
                if feature not in X_pred.columns:
                    X_pred[feature] = 0.0
            
            # Reorder columns to match training data
            X_pred = X_pred[feature_names]
            
            # Handle any missing values
            X_pred = X_pred.fillna(0)
            
            # Scale features
            X_pred_scaled = scaler.transform(X_pred)
            
            # Make predictions
            y_pred = model.predict(X_pred_scaled)
            
            # Update predictions DataFrame
            pred_df = pd.DataFrame({
                'Ticker': valid_features['Ticker'],
                'Predicted_Return': y_pred
            })
            
            # Merge predictions with the full universe
            predictions_df = pd.merge(predictions_df, pred_df, on='Ticker', how='left')
            predictions_df['Predicted_Return'] = predictions_df['Predicted_Return_y'].fillna(predictions_df['Predicted_Return_x'])
            predictions_df = predictions_df[['Ticker', 'Predicted_Return']]
    
    # Calculate signal rank (percentile ranking)
    predictions_df['Signal_Rank'] = predictions_df['Predicted_Return'].rank(pct=True)
    
    return predictions_df

# Create a small sample of today's data and universe to test the function
sample_date = features_df['Date'].max()
sample_sentiment = sentiment_data[sentiment_data['Date'] == sample_date].sample(n=50, random_state=42)
sample_universe = features_df[features_df['Date'] == sample_date]['Ticker'].unique().tolist()[:20]

print(f"Sample date: {sample_date}")
print(f"Sample universe size: {len(sample_universe)}")
print(f"Sample sentiment data size: {len(sample_sentiment)}")

# Test the prediction function
sample_predictions = predict_returns_demo(model_info, sample_sentiment, sample_universe)
print("\nSample predictions:")
print(sample_predictions.head(10))

# Part 5: Conclusions and Insights
print("\n--- Conclusions and Insights ---")

print("1. Model Performance:")
for name, metrics in results.items():
    print(f"   - {name}: Sharpe = {metrics['sharpe']:.4f}, Directional Accuracy = {metrics['directional_acc']:.4f}")

print("\n2. Key Features:")
if 'feature_importance' in locals():
    top_3_features = feature_importance.nlargest(3)
    for feature, importance in top_3_features.items():
        print(f"   - {feature}: {importance:.4f}")

print("\n3. Trading Strategy Potential:")
best_metrics = results[best_model_name]
annual_return = (1 + best_metrics['port_return']) ** 252 - 1
print(f"   - Estimated Annual Return: {annual_return:.2%}")
print(f"   - Daily Sharpe Ratio: {best_metrics['sharpe']:.4f}")
print(f"   - Annualized Sharpe Ratio: {best_metrics['sharpe'] * np.sqrt(252):.4f}")

print("\nNext Steps:")
print("1. Further feature engineering exploring interaction terms")
print("2. Ensemble methods combining multiple model predictions")
print("3. Hyperparameter tuning for optimal model performance")
print("4. Incorporating time-series specific modeling techniques")

Loading data...


KeyboardInterrupt: 